In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection

import tensorflow as tf

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
import utils.custom_tf

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
# type of noise
# asym: classes flip to a single other class
# sym: classes flip uniformly to any other class
TYPE = CONFIG["experiment_configs"][DATASET]["type"]
 # chance of flip
NOISE_P = CONFIG["experiment_configs"][DATASET]["noise_p"]

EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y, IMAGE_X)

print(RANDOM_SEED, TYPE, NOISE_P)

# folders for processed, models
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, DATASET, ".npz", t=TYPE, np=NOISE_P)
# mt = model_type
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base", t=TYPE, np=NOISE_P)

if os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: model has been run for rs={RANDOM_SEED}_t={TYPE}_np={NOISE_P}")

25 asym 0.6


In [4]:
with open(PROCESSED_SAVEPATH, 'rb') as f:
    dat = np.load(f)

    x_train = dat['x_train']
    y_train = dat['y_train']

    x_hyper_train = dat['x_hyper_train']
    y_hyper_train = dat['y_hyper_train']
    
    x_val = dat['x_val']
    y_val = dat['y_val']
    
    x_hyper_val = dat['x_hyper_val']
    y_hyper_val = dat['y_hyper_val']

    x_test = dat['x_test']
    y_test = dat['y_test']

x_val_full = np.concatenate([x_val, x_hyper_val])
y_val_full = np.concatenate([y_val, y_hyper_val])

In [5]:
y_train = tf.keras.utils.to_categorical(y_train)
y_hyper_train = tf.keras.utils.to_categorical(y_hyper_train)
y_val_full = tf.keras.utils.to_categorical(y_val_full)
y_test = tf.keras.utils.to_categorical(y_test)

In [6]:
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE,3),
    nc=10,
)

In [7]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [8]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [9]:
optimizer = tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0)

In [10]:
model.compile(optimizer, loss='categorical_crossentropy', metrics = ['accuracy'])

In [11]:
# this will periodically show the accuracy on valid and test
show_val_test = utils.custom_tf.ShowValidAndTest(
    (x_val_full, y_val_full),
    (x_test, y_test),
    batch_size=BATCH_SIZE,
    epoch_freq=5,
)

def scheduler(epoch):
    if epoch > 80:
        return 0.001
    elif epoch > 40:
        return 0.01
    else:
        return 0.1

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# save best model every epoch
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=BASE_MODEL_SAVEPATH,
    monitor="val_loss",
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

callbacks = [lr_scheduler, show_val_test, save_best]

In [12]:
history = model.fit(
        x=data_generator.flow(
            x_train,
            y_train,
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=RANDOM_SEED,
        ),
        epochs=EPOCHS,
        validation_data=(x_hyper_train, y_hyper_train),
        verbose=1,
        callbacks=callbacks,
)

Epoch 1/120
297/297 [==============================] - ETA: 0s - loss: 1.7624 - accuracy: 0.3996

valid: 0.2865
test: 0.2995



Epoch 00001: val_loss improved from inf to 2.37777, saving model to ../../../models/cifar10/rs=25/cifar10_mt=base_np=0.6_t=asym.h5
297/297 [==============================] - 26s 87ms/step - loss: 1.7624 - accuracy: 0.3996 - val_loss: 2.3778 - val_accuracy: 0.3327
Epoch 2/120
297/297 [==============================] - ETA: 0s - loss: 1.4263 - accuracy: 0.5226
Epoch 00002: val_loss improved from 2.37777 to 2.26452, saving model to ../../../models/cifar10/rs=25/cifar10_mt=base_np=0.6_t=asym.h5
297/297 [==============================] - 24s 82ms/step - loss: 1.4263 - accuracy: 0.5226 - val_loss: 2.2645 - val_accuracy: 0.4480
Epoch 3/120
297/297 [==============================] - ETA: 0s - loss: 1.2793 - accuracy: 0.5709
Epoch 00003: val_loss improved from 2.26452 to 1.71165, saving model to ../../../models/cifar10/rs=25/cifar10_mt=base_np=0.6_t=asym.h5
297/297 [==

Epoch 54/120
297/297 [==============================] - ETA: 0s - loss: 0.6071 - accuracy: 0.7664
Epoch 00054: val_loss did not improve from 0.74097
297/297 [==============================] - 25s 83ms/step - loss: 0.6071 - accuracy: 0.7664 - val_loss: 0.7614 - val_accuracy: 0.7195
Epoch 55/120
297/297 [==============================] - ETA: 0s - loss: 0.5983 - accuracy: 0.7686
Epoch 00055: val_loss did not improve from 0.74097
297/297 [==============================] - 24s 82ms/step - loss: 0.5983 - accuracy: 0.7686 - val_loss: 0.7689 - val_accuracy: 0.7216
Epoch 56/120
297/297 [==============================] - ETA: 0s - loss: 0.6005 - accuracy: 0.7681

valid: 0.5745
test: 0.575



Epoch 00056: val_loss did not improve from 0.74097
297/297 [==============================] - 25s 83ms/step - loss: 0.6005 - accuracy: 0.7681 - val_loss: 0.7777 - val_accuracy: 0.7149
Epoch 57/120
297/297 [==============================] - ETA: 0s - loss: 0.5910 - accuracy: 0.7714
Epoch 00057: val_loss did 

Epoch 111/120
297/297 [==============================] - ETA: 0s - loss: 0.4874 - accuracy: 0.8087

valid: 0.6075
test: 0.6082



Epoch 00111: val_loss did not improve from 0.74023
297/297 [==============================] - 25s 83ms/step - loss: 0.4874 - accuracy: 0.8087 - val_loss: 0.7577 - val_accuracy: 0.7140
Epoch 112/120
297/297 [==============================] - ETA: 0s - loss: 0.4849 - accuracy: 0.8096
Epoch 00112: val_loss did not improve from 0.74023
297/297 [==============================] - 24s 81ms/step - loss: 0.4849 - accuracy: 0.8096 - val_loss: 0.7583 - val_accuracy: 0.7168
Epoch 113/120
297/297 [==============================] - ETA: 0s - loss: 0.4854 - accuracy: 0.8096
Epoch 00113: val_loss did not improve from 0.74023
297/297 [==============================] - 24s 81ms/step - loss: 0.4854 - accuracy: 0.8096 - val_loss: 0.7556 - val_accuracy: 0.7171
Epoch 114/120
297/297 [==============================] - ETA: 0s - loss: 0.4836 - accuracy: 0.8084
Epoch 00114: val_loss

In [13]:
# load best model
model.load_weights(BASE_MODEL_SAVEPATH)

In [14]:
# val acc
preds_val = utils.utils.compute_preds(
    model,
    x_val_full,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_val, axis=1) == np.argwhere(y_val_full)[:,1]).mean()

0.557

In [15]:
# test acc
preds_test = utils.utils.compute_preds(
    model,
    x_test,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_test, axis=1) == np.argwhere(y_test)[:,1]).mean()

0.5588